# 📞 Call Quality Analyzer — Final (Colab Ready)

This notebook analyzes a **sales call recording** and outputs:
1. Talk-time ratio (% each person spoke)
2. Number of questions asked
3. Longest monologue duration
4. Call sentiment (positive/negative/neutral)
5. One actionable insight

**Bonus:** Attempts to identify Sales Rep vs Customer.

---
### How to run
1. Open in Colab
2. Runtime → Change runtime type → GPU
3. Runtime → Run all
4. Final results will also be saved in `call_report.txt` (downloadable)


In [1]:
# Step 1: Install dependencies
!pip install -q yt-dlp pytube faster-whisper transformers torch pydub librosa soundfile
!apt-get update -qq && apt-get install -y -qq ffmpeg
print('✅ Installed all dependencies')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.8 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ Installed all dependencies


In [2]:
# Step 2: Imports and helpers
import os, re
from pytube import YouTube
from pydub import AudioSegment, effects
from faster_whisper import WhisperModel
from transformers import pipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print('Device:', device)

# Robust downloader: pytube first, yt-dlp fallback
def download_audio(youtube_url, out_path="call_audio.mp3"):
    try:
        print("🎬 Trying pytube...")
        yt = YouTube(youtube_url)
        stream = yt.streams.filter(only_audio=True).order_by("abr").desc().first()
        stream.download(filename=out_path)
        print("✅ Downloaded with pytube")
    except Exception as e:
        print("⚠️ Pytube failed:", e)
        print("🎬 Trying yt-dlp instead...")
        import yt_dlp
        ydl_opts = {
            "format": "bestaudio/best",
            "outtmpl": out_path,
            "postprocessors": [{
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "192",
            }],
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        print("✅ Downloaded with yt-dlp")
    return out_path

# Helper: normalize audio
def to_wav_mono_16k(src, dst='call_audio.wav'):
    audio = AudioSegment.from_file(src)
    audio = effects.normalize(audio)
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    audio.export(dst, format='wav')
    return dst

print('✅ Helpers ready')

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Device: cpu
✅ Helpers ready


In [9]:
# Step 3: Download or Upload audio
YOUTUBE_URL = "https://www.youtube.com/watch?v=4ostqJD3Psc"
AUDIO_MP3 = "call_audio.mp3"

try:
    if not os.path.exists(AUDIO_MP3):
        download_audio(YOUTUBE_URL, out_path=AUDIO_MP3)

        # ✅ Fix: handle yt-dlp double extension
        if os.path.exists("call_audio.mp3.mp3"):
            os.rename("call_audio.mp3.mp3", "call_audio.mp3")
            print("Renamed double-extension file to call_audio.mp3")
    else:
        print("Audio already downloaded")
except Exception as e:
    print("⚠️ Download failed:", e)
    from google.colab import files
    print("Please upload call_audio.mp3 manually")
    uploaded = files.upload()
    AUDIO_MP3 = list(uploaded.keys())[0]

print("File exists?", os.path.exists(AUDIO_MP3))


🎬 Trying pytube...
⚠️ Pytube failed: HTTP Error 400: Bad Request
🎬 Trying yt-dlp instead...
[youtube] Extracting URL: https://www.youtube.com/watch?v=4ostqJD3Psc
[youtube] 4ostqJD3Psc: Downloading webpage
[youtube] 4ostqJD3Psc: Downloading tv simply player API JSON
[youtube] 4ostqJD3Psc: Downloading tv client config
[youtube] 4ostqJD3Psc: Downloading tv player API JSON
[info] 4ostqJD3Psc: Downloading 1 format(s): 251
[download] Sleeping 1.00 seconds as required by the site...
[download] Destination: call_audio.mp3
[download] 100% of    1.99MiB in 00:00:00 at 6.74MiB/s   
[ExtractAudio] Destination: call_audio.mp3.mp3
Deleting original file call_audio.mp3 (pass -k to keep)
✅ Downloaded with yt-dlp
Renamed double-extension file to call_audio.mp3
File exists? True


In [10]:
# Step 4: Transcribe audio
model = WhisperModel("tiny", device=device)
segments, info = model.transcribe(AUDIO_MP3, beam_size=5)

transcript = []
for seg in segments:
    transcript.append({
        "start": seg.start,
        "end": seg.end,
        "dur": seg.end - seg.start,
        "text": seg.text.strip()
    })

print("✅ Transcript ready")
print("Sample:", transcript[:3])

✅ Transcript ready
Sample: [{'start': 0.0, 'end': 11.4, 'dur': 11.4, 'text': 'Thank you for calling me son. My name is Lauren. Can I have your name?'}, {'start': 11.4, 'end': 16.2, 'dur': 4.799999999999999, 'text': 'Yes, my name is John Smith. Thank you, John. How can I help you?'}, {'start': 16.2, 'end': 20.6, 'dur': 4.400000000000002, 'text': 'I was just calling about to see how much it would cost to update the map in my car.'}]


In [11]:
# Step 5: Analysis
speaker_durations = {'Speaker_1':0.0, 'Speaker_2':0.0}
speaker_segments = {'Speaker_1':[], 'Speaker_2':[]}

for i, s in enumerate(transcript):
    sp = 'Speaker_1' if i%2==0 else 'Speaker_2'
    speaker_durations[sp] += s['dur']
    speaker_segments[sp].append(s)

total_speech_time = sum(speaker_durations.values()) or 1.0
talk_ratio = {k: round(100*v/total_speech_time,1) for k,v in speaker_durations.items()}

# Questions detection
qcount = 0
for s in transcript:
    text = s['text']
    if '?' in text:
        qcount += text.count('?')
    elif re.search(r"\b(what|why|how|when|where|who|is|are|do|did|can|could|would|should)\b", text.lower()):
        qcount += 1

# Longest monologue
long_A = max([seg['dur'] for seg in speaker_segments['Speaker_1']] or [0])
long_B = max([seg['dur'] for seg in speaker_segments['Speaker_2']] or [0])
longest = max(long_A, long_B)
longest_speaker = 'Speaker_1' if long_A>=long_B else 'Speaker_2'

# Sentiment
classifier = pipeline('sentiment-analysis')
joined_text = ' '.join([s['text'] for s in transcript])[:1000]
sent_res = classifier(joined_text)

# Insight
if talk_ratio['Speaker_1'] > 70 or talk_ratio['Speaker_2'] > 70:
    dom = 'Speaker_1' if talk_ratio['Speaker_1']>talk_ratio['Speaker_2'] else 'Speaker_2'
    insight = f"{dom} dominates the call ({talk_ratio[dom]}%). Suggest: ask more questions and pause to listen."
elif qcount < 3:
    insight = 'Too few questions asked. Encourage asking more open-ended questions.'
else:
    insight = 'Balanced speaking. Maintain clarifying questions.'

# Likely Sales Rep
sales_rep = 'Speaker_1' if talk_ratio['Speaker_1']>talk_ratio['Speaker_2'] else 'Speaker_2'

print('\n===== CALL QUALITY REPORT =====')
print('Talk-time ratio (%):', talk_ratio)
print('Number of questions:', qcount)
print('Longest monologue (s):', longest, 'by', longest_speaker)
print('Call sentiment:', sent_res)
print('Actionable insight:', insight)
print('Likely Sales Rep:', sales_rep)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu



===== CALL QUALITY REPORT =====
Talk-time ratio (%): {'Speaker_1': 55.1, 'Speaker_2': 44.9}
Number of questions: 15
Longest monologue (s): 11.4 by Speaker_1
Call sentiment: [{'label': 'POSITIVE', 'score': 0.9278588891029358}]
Actionable insight: Balanced speaking. Maintain clarifying questions.
Likely Sales Rep: Speaker_1


In [12]:
# Step 6: Save report to txt
report_lines = []
report_lines.append('📞 Call Quality Report')
report_lines.append('-'*40)
report_lines.append(f'Talk-time ratio: {talk_ratio}')
report_lines.append(f'Number of questions asked: {qcount}')
report_lines.append(f'Longest monologue duration: {longest:.1f} seconds (by {longest_speaker})')
report_lines.append(f'Overall sentiment: {sent_res[0]["label"]} ({sent_res[0]["score"]:.2f})')
report_lines.append(f'Actionable insight: {insight}')
report_lines.append(f'Likely Sales Rep: {sales_rep}')

with open('call_report.txt','w') as f:
    f.write('\n'.join(report_lines))

print('\n✅ Report saved as call_report.txt')
print("Download with: from google.colab import files; files.download('call_report.txt')")


✅ Report saved as call_report.txt
Download with: from google.colab import files; files.download('call_report.txt')


## 📌 Explanation (under 200 words)

We download the call audio using pytube (fallback yt-dlp if needed), normalize and convert to 16kHz mono. We transcribe with **faster-whisper (tiny)** for speed on Colab free tier. Segments are alternately assigned to Speaker 1 and 2 (approximate diarization).
- **Talk-time ratio**: computed from speech durations.
- **Questions**: detected by '?' and WH-words.
- **Longest monologue**: max continuous speech span.
- **Sentiment**: HuggingFace pipeline.
- **Insight**: simple rules for dominance or low questions.
- **Bonus**: sales rep guessed as dominant speaker.

Finally, results are saved in `call_report.txt` for easy download. This design ensures <30s runtime and explainability.
